<a href="https://colab.research.google.com/github/JarvisSan22/JC_learn_python_Notebooks/blob/main/TJC_Style_transfer__TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization?hl=ja

In [2]:
# Google Driveをマウント
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

import tensorflow as tf
import numpy as np
import os
import glob 
import tensorflow as tf
import tensorflow_hub as hub

from matplotlib import gridspec
import matplotlib.pylab as plt

style_files=glob.glob("/content/drive/MyDrive/Colab Notebooks/Style_transfer/Styles/***")
print(f"{'#'*4} styles {'#'*4} ")
for i,f in enumerate(style_files):
  print(i,os.path.basename(f))

#### styles #### 
0 Hokusa_great_wave.jpg
1 見立源氏はなの宴.jpg
2 Kuniyoshi_fish.jpg
3 bucchibi_yoruhamitaimonogatakusan.jpg
4 草間 彌生.jpg
5 杉浦非水_1.jpg
6 杉浦非水_2.jpg
7 Kuniyoshi_fish_anime_red.jpg
8 Kuniyoshi_fish_2.jpg


In [6]:

def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

def upload_file(): 
  from google.colab import files 
  uploaded = files.upload() 
  for k, v in uploaded.items(): 
    open(k, 'wb').write(v) 
  return list(uploaded.keys())[0] 


def load_image(image_path=None, image_size=(256, 256), preserve_aspect_ratio=True):
  if not image_path:
    image_path=upload_file()
  img= plt.imread(image_path).astype(np.float32)
  if img.max() > 1.0:
    img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w  * n, w))

  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()

In [7]:
hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

In [11]:
content_path=None
if not content_path:
  content_path=upload_file()


Saving Castle_1.jpg to Castle_1.jpg


In [ ]:
style_n=1
if not style_n:
  style_path=upload_file()
else:
  style_path=style_files[style_n]


content_image = load_image(content_path)
style_image = load_image(style_path)
style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
stylized_image = outputs[0]

show_n([content_image, style_image, stylized_image], 
       titles=['Original content image', 'Style image', 'Stylized image'])

